##Clustering based on $\sigma_{\epsilon}^2$

In [1]:
import numpy as np
import scipy as sp
import scipy.special
import scipy.linalg
%pylab
%matplotlib inline
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [41]:
trials = 1

# Generate complex-valued Gaussian random vector
k = linspace(0,100,3000)
K = k.shape[0]

# Real part is an oscillation with period K0
K0 = 12.5
c1m = cos(2*pi*(k-((K0)/4))/(K0))

# Imaginary part is linear
#c2m = k/100
c2m = cos(2*pi*(k-(8*K0/4))/(8*K0))

# normalize cluster 1 mean and culster 2 mean they have the same energy
#c2m = sqrt(var(c1m)/var(c2m))*c2m

# Add Gaussian noise (based on real part, which is 0 mean)
snr = 10 # in dB

sigma2e = var(c1m) + var(c2m)

sigma2v = sigma2e*10**(-snr/10)

y1 = c1m + sqrt(sigma2v)*randn(K) 
y2 = c2m + sqrt(sigma2v)*randn(K)

# Generate 10 time series for each cluster

Y1 = c1m + sqrt(sigma2v)*randn(10,K)

Y2 = c2m + sqrt(sigma2v)*randn(10,K)

Y = concatenate((Y1,Y2),axis=0)


In [13]:
def Do_Kalman_Likelihood(y, sigma2obs, sigma2e):
    """MAP solution, inverse covariance matrix, and marginal loglikelihood of state-space model

    :param y: Observations (K,)
    :param sigma2obs: Variance of observation noise (can be scalar or vector)
    :param sigma2e: Variance of process noise
    :return: x_map, L, marginal_loglikelihood, joint_loglikelihood
    """
    # Build diagonals of information matrix
    sigma2obs *= np.ones(len(y))
    D = 1. / sigma2obs + 2. / sigma2e
    D[-1] = 1. / sigma2obs[-1] + 1. / sigma2e
    B = -np.ones(len(D)) / sigma2e
    B[0] = 0.
    
    # Solve, assuming x_init=0 for simplicity
    #L = sp.linalg.cholesky_banded((D, B), lower=True)
    U = sp.linalg.cholesky_banded((B, D), lower=False)

    x_map = sp.linalg.cho_solve_banded([U, False], y / sigma2obs)

    # Compute joint and marginal probabilities
    joint_loglikelihood = -.5 * ((np.sum(np.diff(x_map)**2) + x_map[0]**2) / sigma2e +
                                 np.sum((y - x_map)**2 / sigma2obs) +
                                 (len(y) * np.log(2*np.pi*sigma2e * 2*np.pi) + np.sum(np.log(sigma2obs))))
    marginal_loglikelihood = len(y)/2. * np.log(2*np.pi) + joint_loglikelihood - np.sum(np.log(U[-1]))
    return x_map, U, marginal_loglikelihood, joint_loglikelihood

In [14]:
def cov_from_chol_precision(U):
    """Given the Cholesky factorization (U) of the posterior precision matrix (J), with U^t * U = J,
    return the tridiagonal part of the covariance matrix.

    :param U: Cholesky factorization (U) of J, given as [0, A; D] where A is the upper diagonal and D the main diagonal
    :return: Cov_tri: Tridiagonal part of the covariance matrix returned as [0, C_i,i+1; C_ii; C_i+1,i, 0]
    """
    assert(U.shape[0] == 2 and U[0,0] == 0)
    A, D = U # Unpack matrix into first (above) diagonal and diagonal
    Cov_tri = np.zeros_like(U)
    C, V = Cov_tri # Obtain _views_ into the first diagonal and diagonal
    # Compute last element of diagonal
    V[-1] = 1. / (D[-1] ** 2)
    # Recursively compute other elements of main diagonal and first diagonal
    for i in range(len(D)-1, 0, -1):
        iD = 1. / D[i-1]
        iDA = iD * A[i]
        N = -iDA * V[i]
        C[i] = N
        V[i-1] = iD ** 2 - N * iDA
    return Cov_tri

In [22]:
def Cluster_Kalman(Y, sigma2obs, sigma2e_init, prior_clusters, trials=1., verbose=True):
    assert(len(sigma2e_init) == len(prior_clusters))
    C = len(prior_clusters)
    M, K = Y.shape
    posterior_clusters = np.tile(prior_clusters, [M,1]).T
    sigma2e = sigma2e_init
    expect_log_likel_old = np.NaN
    while True:
        sigma2e_new = np.zeros((C, M))
        log_prob_Yi_given_c = np.zeros((C, M))
        for i, Y_i in enumerate(Y):
            for c, sigma2e_c in enumerate(sigma2e):
                x_map, U, marginal_loglik, _ = Do_Kalman_Likelihood(Y_i, sigma2obs, sigma2e_c)
                Cov_tri = cov_from_chol_precision(U)
                sigma2e_new[c,i] = (np.sum(Cov_tri[1]) + np.dot(x_map, x_map) # E[x_k^2]
                                   + np.sum(Cov_tri[1,:-1]) + np.dot(x_map[:-1], x_map[:-1]) # E[x_{k-1}^2]
                                   - 2 * np.sum(Cov_tri[0]) - 2 * np.dot(x_map[1:], x_map[:-1])) / K # E[x_{k-1} * x_k]
                log_prob_Yi_given_c[c,i] = marginal_loglik
        expect_log_likel = np.sum(posterior_clusters * log_prob_Yi_given_c)
        if verbose:
            print(expect_log_likel, sigma2e)
        if (abs(expect_log_likel - expect_log_likel_old) < 1e-6 * abs(expect_log_likel_old)):
            break
        expect_log_likel_old = expect_log_likel
        sigma2e = np.sum(posterior_clusters * sigma2e_new, axis=1) / np.sum(posterior_clusters, axis=1)
        posterior_clusters = np.exp(log_prob_Yi_given_c - np.max(log_prob_Yi_given_c, axis=0)) * prior_clusters[:,None]
        posterior_clusters /= np.sum(posterior_clusters, axis=0)
        prior_clusters = np.sum(posterior_clusters, axis=1) / np.sum(posterior_clusters, axis=None)
    return sigma2e, prior_clusters, posterior_clusters, expect_log_likel

In [70]:
%%time
#sigma2e_init = np.array([0.01, 0.001])
sigma2e_init = np.array([0.0001, 0.001])
#sigma2e_init = np.array([0.001, 0.01])


prior_clusters = np.array([0.5, 0.5])
sigma2e, prior_clusters, posterior_clusters, expect_log_likel = Cluster_Kalman(Y, sigma2v, sigma2e_init, prior_clusters, trials=trials)
print('---- Expectation of log_likelihood -----')
print(expect_log_likel)
print('---- Sigma2e -----')
print(sigma2e)
print('---- Prior clusters (estimated) -----')
print(prior_clusters)
print('---- Posterior clusters -----')
print(posterior_clusters)

(-26392.513675610204, array([ 0.0001,  0.001 ]))
(-19379.407336604643, array([ 0.00014199,  0.00104147]))
(-19243.851122904016, array([ 0.0001428 ,  0.00114598]))
(-19138.158549663822, array([ 0.0001436 ,  0.00124699]))
(-19054.521891931432, array([ 0.00014439,  0.00134433]))
(-18987.520458421655, array([ 0.00014516,  0.00143789]))
(-18933.288680523619, array([ 0.00014593,  0.00152761]))
(-18889.006398587288, array([ 0.00014668,  0.00161347]))
(-18852.575482425145, array([ 0.00014743,  0.00169549]))
(-18822.408349469093, array([ 0.00014816,  0.0017737 ]))
(-18797.285932807576, array([ 0.00014889,  0.00184818]))
(-18776.26002541896, array([ 0.0001496,  0.001919 ]))
(-18758.584720571238, array([ 0.0001503 ,  0.00198625]))
(-18743.667374396286, array([ 0.000151  ,  0.00205004]))
(-18731.032940711804, array([ 0.00015168,  0.00211048]))
(-18720.297638106793, array([ 0.00015235,  0.00216769]))
(-18711.149240956678, array([ 0.00015302,  0.00222179]))
(-18703.332144964843, array([ 0.00015367, 

In [75]:
s2e = np.arange(0.00001, 0.005, .0001)
#print s2e
marginal_loglik = np.zeros_like(s2e)
loglik_s1_s2 = np.zeros([len(s2e) , len(s2e)])
col = ['', 'r', 'g', 'b']
for i, Y_i in enumerate(Y):
    for j in range(len(s2e)):
        marginal_loglik[j] = Do_Kalman_Likelihood(Y_i, sigma2v, s2e[j])[2]
    offset = np.max(marginal_loglik)
    marginal_lik = np.exp(marginal_loglik - offset)
    loglik_s1_s2 += offset + np.log(.5 * (marginal_lik[None,:] + marginal_lik[:,None]))
plt.contour(s2e, s2e, loglik_s1_s2)
plot(sigma2e[0],sigma2e[1],'ro')

TypeError: object of type 'numpy.float64' has no len()

In [74]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = figure()
ax = fig.gca(projection='3d')
S1e, S2e = np.meshgrid(s2e, s2e)

surf = ax.plot_surface(S1e, S2e, loglik_s1_s2, rstride=1, cstride=1)#, cmap=cm.Accent,
                       #linewidth=0, antialiased=False)
    
plot(sigma2e[0],sigma2e[1],'ro')

#ax.set_zlim(-25000, -18000)

#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

#fig.colorbar(surf, shrink=0.5, aspect=5)

TypeError: object of type 'numpy.float64' has no len()

In [77]:
print unravel_index(np.argmax(loglik_s1_s2),loglik_s1_s2.shape)

(2, 30)


In [79]:
print s2e[2], s2e[30]

0.00021 0.00301
